In [ ]:
import os
import itertools
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16, VGG19, MobileNetV3Large, ResNet152V2, MobileNet
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_HEIGHT = 224
IMG_WIDTH = 224
NUM_CLASSES = 7
BATCH_SIZE = 32
EPOCHS = 10  # Increase the number of epochs for better training

data_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Function to create MobileNet model
def create_mobilenet_v3_large():
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    # Fine-tune only the last convolutional block
    for layer in base_model.layers[:-4]:
        layer.trainable = False

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)  # Add dropout layer
    predictions = layers.Dense(NUM_CLASSES, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=predictions)
    return model

# Function to create VGG16 model
def create_vgg16():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    # Fine-tune only the last convolutional block
    for layer in base_model.layers[:-4]:
        layer.trainable = False

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)  # Add dropout layer
    predictions = layers.Dense(NUM_CLASSES, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=predictions)
    return model

# Function to create VGG19 model
def create_vgg19():
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    # Fine-tune only the last convolutional block
    for layer in base_model.layers[:-4]:
        layer.trainable = False

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)  # Add dropout layer
    predictions = layers.Dense(NUM_CLASSES, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=predictions)
    return model

# Function to create ResNet152V2 model
def create_resnet152v2():
    base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    # Fine-tune only the last convolutional block
    for layer in base_model.layers[:-4]:
        layer.trainable = False

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)  # Add dropout layer
    predictions = layers.Dense(NUM_CLASSES, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=predictions)
    return model

def lr_scheduler(epoch, lr):
    if epoch < 100:
        return 0.001  # Keep the initial learning rate for the first 10 epochs
    else:
        return lr  # Keep the learning rate constant after the 10th epoch

def train_without_early_stopping(model, train_gen, val_gen, epochs, model_name):
    history = model.fit(
        train_gen,
        steps_per_epoch=train_gen.samples // BATCH_SIZE,
        epochs=epochs,
        validation_data=val_gen,
        validation_steps=val_gen.samples // BATCH_SIZE
    )
    return history


# Function to extract F1-score, precision, and recall from the classification report
def extract_metrics(classification_rep):
    class_names = list(validation_generator.class_indices.keys())
    f1_scores = [classification_rep[emotion]['f1-score'] for emotion in class_names]
    precisions = [classification_rep[emotion]['precision'] for emotion in class_names]
    recalls = [classification_rep[emotion]['recall'] for emotion in class_names]
    return f1_scores, precisions, recalls

# Function to calculate metrics (F1-score, precision, and recall)
def calculate_metrics(model, test_generator):
    y_true = test_generator.classes
    y_pred = model.predict(test_generator)
    y_pred = np.argmax(y_pred, axis=1)
    # Calculate confusion matrix
    confusion_mat = confusion_matrix(y_true, y_pred)
    # Calculate classification report
    class_names = list(validation_generator.class_indices.keys())
    classification_rep = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    return confusion_mat, classification_rep

# Function to plot multiple confusion matrices
def plot_multiple_confusion_matrices_values(confusion_mats, model_names, emotions):
    plt.figure(figsize=(10, 8))

    for i, confusion_mat in enumerate(confusion_mats):
        total_samples = np.sum(confusion_mat)
        confusion_mat_percentage = confusion_mat / total_samples * 100  # Convert to percentage

        plt.subplot(2, 2, i + 1)
        plt.imshow(confusion_mat_percentage, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title(f'Confusion Matrix - {model_names[i]}')
        plt.colorbar()
        tick_marks = np.arange(len(emotions))
        plt.xticks(tick_marks, emotions, rotation=45)
        plt.yticks(tick_marks, emotions)
        plt.xlabel('Predicted Emotion')
        plt.ylabel('True Emotion')

        for j, k in itertools.product(range(confusion_mat.shape[0]), range(confusion_mat.shape[1])):
            plt.text(k, j, f'{confusion_mat_percentage[j, k]:.2f}', horizontalalignment="center", color="white" if confusion_mat_percentage[j, k] > 50 else "black")

    plt.tight_layout()
    plt.show()



# Function to plot the confusion matrix for multiple models
def plot_multiple_confusion_matrices(confusion_mats, model_names, emotions):
    plt.figure(figsize=(10, 8))
    for i, confusion_mat in enumerate(confusion_mats):
        plt.subplot(2, 2, i + 1)
        plt.imshow(confusion_mat, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title(f'Confusion Matrix - {model_names[i]}')
        plt.colorbar()
        tick_marks = np.arange(len(emotions))
        plt.xticks(tick_marks, emotions, rotation=45)
        plt.yticks(tick_marks, emotions)
        plt.xlabel('Predicted Emotion')
        plt.ylabel('True Emotion')

        thresh = confusion_mat.max() / 2.0
        for j, k in itertools.product(range(confusion_mat.shape[0]), range(confusion_mat.shape[1])):
            plt.text(k, j, format(confusion_mat[j, k], 'd'), horizontalalignment="center", color="white" if confusion_mat[j, k] > thresh else "black")

    plt.tight_layout()
    plt.show()

# Function to plot training history (loss and accuracy) for all models
def plot_training_history(model_histories, model_names, model_colors):
    num_models = len(model_histories)

    plt.figure(figsize=(12, 10))

    # Plot Validation Loss
    plt.subplot(2, 2, 1)
    for i, (history, model_name) in enumerate(zip(model_histories, model_names)):
        plt.plot(history.history['val_loss'], label=model_name, color=model_colors[i])
    plt.xlabel('Epoch')
    plt.ylabel('Validation Loss')
    plt.title('Validation Loss')
    plt.legend()
    plt.grid(True)

    # Plot Training Loss
    plt.subplot(2, 2, 2)
    for i, (history, model_name) in enumerate(zip(model_histories, model_names)):
        plt.plot(history.history['loss'], label=model_name, color=model_colors[i])
    plt.xlabel('Epoch')
    plt.ylabel('Training Loss')
    plt.title('Training Loss')
    plt.legend()
    plt.grid(True)

    # Plot Validation Accuracy
    plt.subplot(2, 2, 3)
    for i, (history, model_name) in enumerate(zip(model_histories, model_names)):
        plt.plot(history.history['val_accuracy'], label=model_name, color=model_colors[i])
    plt.xlabel('Epoch')
    plt.ylabel('Validation Accuracy')
    plt.title('Validation Accuracy')
    plt.legend()
    plt.grid(True)

    # Plot Training Accuracy
    plt.subplot(2, 2, 4)
    for i, (history, model_name) in enumerate(zip(model_histories, model_names)):
        plt.plot(history.history['accuracy'], label=model_name, color=model_colors[i])
    plt.xlabel('Epoch')
    plt.ylabel('Training Accuracy')
    plt.title('Training Accuracy')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Set the random seed for reproducibility
tf.random.set_seed(42)

train_data_dir = '/kaggle/input/student-facial-expression-recognition/SFER dataset/SFER dataset/train'  # Replace with the path to your "train" folder
test_data_dir = '/kaggle/input/student-facial-expression-recognition/SFER dataset/SFER dataset/test'    # Replace with the path to your "test" folder

train_generator = data_generator.flow_from_directory(
    train_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = data_generator.flow_from_directory(
    train_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

test_generator = data_generator.flow_from_directory(
    test_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Define class names
class_names = list(train_generator.class_indices.keys())

# Create models
mobilenet_v3_large_model = create_mobilenet_v3_large()
vgg16_model = create_vgg16()
vgg19_model = create_vgg19()
resnet152v2_model = create_resnet152v2()

# Compile models
mobilenet_v3_large_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg19_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet152v2_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks to save best weights
mobilenet_checkpoint = ModelCheckpoint('mobilenet_best1_weights.h5', save_best_only=True)
vgg16_checkpoint = ModelCheckpoint('vgg16_best1_weights.h5', save_best_only=True)
vgg19_checkpoint = ModelCheckpoint('vgg19_best1_weights.h5', save_best_only=True)
resnet152v2_checkpoint = ModelCheckpoint('resnet152v2_best1_weights.h5', save_best_only=True)

# Train models with the checkpoint callbacks
mobilenet_history = mobilenet_v3_large_model.fit(
    train_generator, 
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,  
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[mobilenet_checkpoint]
)

vgg16_history = vgg16_model.fit(
    train_generator, 
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,  
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[vgg16_checkpoint]
)

vgg19_history = vgg19_model.fit(
    train_generator, 
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator, 
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[vgg19_checkpoint]
)

resnet152v2_history = resnet152v2_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,  
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[resnet152v2_checkpoint]
)

# Load best weights for evaluation
mobilenet_v3_large_model.load_weights('mobilenet_best1_weights.h5')
vgg16_model.load_weights('vgg16_best1_weights.h5')
vgg19_model.load_weights('vgg19_best1_weights.h5')
resnet152v2_model.load_weights('resnet152v2_best1_weights.h5')

# Function to calculate mean metrics
def calculate_mean_metrics(history):
    mean_val_loss = np.mean(history.history['val_loss'])
    mean_train_loss = np.mean(history.history['loss'])
    mean_val_acc = np.mean(history.history['val_accuracy'])
    mean_train_acc = np.mean(history.history['accuracy'])
    return mean_val_loss, mean_train_loss, mean_val_acc, mean_train_acc


# Calculate metrics for all models and save to the list
mobilenet_metrics = calculate_mean_metrics(mobilenet_history)
vgg16_metrics = calculate_mean_metrics(vgg16_history)
vgg19_metrics = calculate_mean_metrics(vgg19_history)
resnet152v2_metrics = calculate_mean_metrics(resnet152v2_history)


# Calculate metrics for all models
mobilenet_confusion_mat, mobilenet_classification_rep = calculate_metrics(mobilenet_v3_large_model, test_generator)
vgg16_confusion_mat, vgg16_classification_rep = calculate_metrics(vgg16_model, test_generator)
vgg19_confusion_mat, vgg19_classification_rep = calculate_metrics(vgg19_model, test_generator)
resnet152v2_confusion_mat, resnet152v2_classification_rep = calculate_metrics(resnet152v2_model, test_generator)

# Extract F1-score, precision, and recall for all models
mobilenet_f1, mobilenet_precision, mobilenet_recall = extract_metrics(mobilenet_classification_rep)
vgg16_f1, vgg16_precision, vgg16_recall = extract_metrics(vgg16_classification_rep)
vgg19_f1, vgg19_precision, vgg19_recall = extract_metrics(vgg19_classification_rep)
resnet152v2_f1, resnet152v2_precision, resnet152v2_recall = extract_metrics(resnet152v2_classification_rep)

# Define colors for bars
mobilenet_color = 'limegreen'
vgg16_color = 'dodgerblue'
vgg19_color = 'darkorange'
resnet152v2_color = 'orchid'  # Add a new color for ResNet152v2

# Define model names for the bar plot
model_names = ['MobileNet', 'VGG16', 'VGG19', 'ResNet152V2']

# Plot F1-score, precision, recall, and accuracy for all models on a single graph
plt.figure(figsize=(16, 12))

bar_width = 0.15
bar_positions = np.arange(len(class_names))

# Plot F1-score
plt.subplot(2, 2, 1)
plt.bar(bar_positions - 1.5 * bar_width, mobilenet_f1, width=bar_width, label='MobileNet', align='center', color=mobilenet_color)
plt.bar(bar_positions - 0.5 * bar_width, vgg16_f1, width=bar_width, label='VGG16', align='center', color=vgg16_color)
plt.bar(bar_positions + 0.5 * bar_width, vgg19_f1, width=bar_width, label='VGG19', align='center', color=vgg19_color)
plt.bar(bar_positions + 1.5 * bar_width, resnet152v2_f1, width=bar_width, label='ResNet152V2', align='center', color=resnet152v2_color)
plt.xticks(bar_positions, class_names)
plt.xlabel('Emotion')
plt.ylabel('F1-score')
plt.title('F1-score Comparison')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Plot Precision
plt.subplot(2, 2, 2)
plt.bar(bar_positions - 1.5 * bar_width, mobilenet_precision, width=bar_width, label='MobileNet', align='center', color=mobilenet_color)
plt.bar(bar_positions - 0.5 * bar_width, vgg16_precision, width=bar_width, label='VGG16', align='center', color=vgg16_color)
plt.bar(bar_positions + 0.5 * bar_width, vgg19_precision, width=bar_width, label='VGG19', align='center', color=vgg19_color)
plt.bar(bar_positions + 1.5 * bar_width, resnet152v2_precision, width=bar_width, label='ResNet152V2', align='center', color=resnet152v2_color)
plt.xticks(bar_positions, class_names)
plt.xlabel('Emotion')
plt.ylabel('Precision')
plt.title('Precision Comparison')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Plot Recall
plt.subplot(2, 2, 3)
plt.bar(bar_positions - 1.5 * bar_width, mobilenet_recall, width=bar_width, label='MobileNet', align='center', color=mobilenet_color)
plt.bar(bar_positions - 0.5 * bar_width, vgg16_recall, width=bar_width, label='VGG16', align='center', color=vgg16_color)
plt.bar(bar_positions + 0.5 * bar_width, vgg19_recall, width=bar_width, label='VGG19', align='center', color=vgg19_color)
plt.bar(bar_positions + 1.5 * bar_width, resnet152v2_recall, width=bar_width, label='ResNet152V2', align='center', color=resnet152v2_color)
plt.xticks(bar_positions, class_names)
plt.xlabel('Emotion')
plt.ylabel('Recall')
plt.title('Recall Comparison')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Plot Accuracy
plt.subplot(2, 2, 4)
model_accuracies = [mobilenet_history.history['val_accuracy'][-1],
                    vgg16_history.history['val_accuracy'][-1],
                    vgg19_history.history['val_accuracy'][-1],
                    resnet152v2_history.history['val_accuracy'][-1]]

# Adjust the width of the bars to make them thicker
bar_width = 0.2 # You can increase this value to make the bars thicker
plt.bar(model_names, model_accuracies, width=bar_width, color=[mobilenet_color, vgg16_color, vgg19_color, resnet152v2_color])

plt.xlabel('Model')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy Comparison')
plt.grid(axis='y', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# Plot the confusion matrix for all models
confusion_mats = [mobilenet_confusion_mat, vgg16_confusion_mat, vgg19_confusion_mat, resnet152v2_confusion_mat]
model_names = ['MobileNet', 'VGG16', 'VGG19', 'ResNet152V2']
plot_multiple_confusion_matrices_values(confusion_mats, model_names, emotions=class_names)

# Plot the training history for all models
model_colors = ['limegreen', 'dodgerblue', 'darkorange', 'orchid']  # Update with the colors you prefer
model_histories = [mobilenet_history, vgg16_history, vgg19_history, resnet152v2_history]
plot_training_history(model_histories, model_names, model_colors)



Found 5343 images belonging to 7 classes.
Found 1334 images belonging to 7 classes.
Found 1399 images belonging to 7 classes.
17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
